# Exact Diagonalization of $J$-$K$ Hamiltonian
    Alan Morningstar
    May 2017

## Include Source

In [1]:
include("utils.jl");
include("lattice.jl");
include("basis.jl");
include("sparseHam.jl");

## Main

#### specify parameters

In [2]:
# square lattice length
const Lx = 4;
const Ly = 4;
# number of sites
const N = Lx*Ly;
# NN coupling
const J1 = 1.0;
# plaquette coupling
const K = 0.0;

# choose Sz sector by specifying number of 1s in basis states
const n1 = convert(Int,N/2);
# choose kx,ky by specifying mi such that mi is in 0:Li-1
const mx = 0;
const my = 0;
const kx = 2*pi*mx/Lx;
const ky = 2*pi*my/Ly;

# number of eigenvalues desired
const numEigs = 4;
# a tolerance for error
const tolerance = 10.^(-5.);
# ritzVec = true if you want the eigenvectors returned too
const ritzVec = false;
# number of Krylov vectors in eigenvalue calculation
const numKrylovVecs = 20;
# maximum number of iterations to converge eigenvalues
const maxIter = 300;

# plaquette (x,y) vectors, locating p1,p2,p3,p4 on the plaquette of the p1 site and p1D,p2D,p1L,p3L on adjacent plaquettes
const neighborVectors = [(0,0),(1,0),(0,1),(1,1),(0,-1),(1,-1),(-1,0),(-1,1)];

#### define lattice, symmetry sector, and basis

In [3]:
# define the lattice
const l = lattice(Lx,Ly,neighborVectors);

In [4]:
# specify symmetry sector
const s = sector(n1,kx,ky);

In [5]:
# construct the basis
# 0.49 seconds for 6x4 lattice, basis is ~0.002 GB
@time const basis = SzkxkyBasis(l,s);
println("Dimension of reduced Hilbert space is ",basis.dim,".");

  0.137826 seconds (146.77 k allocations: 9.325 MB, 6.99% gc time)
Dimension of reduced Hilbert space is 822.


#### build the Hamiltonian

In [6]:
# couplings type to make passing J1,K easier
const c = couplings(J1,K);

# build the sparse Hamiltonian
# takes 3.3 seconds for 6x4 full J1-K Heisenberg model, H contains ~0.16 GB
@time const H = constructSparseHam(basis,c,s,l);

  0.506932 seconds (574.41 k allocations: 22.924 MB, 1.04% gc time)


In [7]:
# using ProfileView;

In [8]:
# Profile.clear();
# @profile constructSparseHam(basis,c,s,l);
# ProfileView.view()

#### find eigenvalues

In [9]:
#- compute eigenvalues
# ~5.7 seconds for 6x4, numEigs=4, which=:SR, full J1-K Heisenberg Hamiltonian
#:LM stands for largest magnitude, :SR for smallest real part
@time eigsResult = eigs(H; nev=numEigs,ncv=numKrylovVecs,maxiter=maxIter, which=:LM, tol=tolerance, ritzvec=ritzVec);

  3.085042 seconds (4.05 M allocations: 164.008 MB, 1.68% gc time)


In [10]:
# print energies
println("Energies are: ",real(eigsResult[1]),".");
# print algorithm performance
println("Number of iterations = ",eigsResult[3]);
println("Number of matrix-vector multiplications = ",eigsResult[4]);

Energies are: [-9.39423,-7.64951,-6.78919,-6.35789].
Number of iterations = 3
Number of matrix-vector multiplications = 48
